## Othello clip training notebook
to use Google's larger machines

In [ ]:
import datetime
import os.path
import time

import clip
import data_handling
from clip import inference
from clip.inference import ClipInference
from clip.model import CLIPModel
#from data_handling.clip_data_prep import ClipDataPreparator
#from data_handling.game_parser import GameParser
#from data_handling.book_parser import BookParser
#from data_handling.common import convert_to_notation
import torch
from othello_game import othello
import data_handling.common as cm
#from clip.train import ClipTrainer
from clip import utils


In [ ]:
def play_game():
    # plays a sample game of othello based on the move string

    cols = "ABCDEFGH"

    #moves = "f5d6c4d3e6f4e3f6c5b4e7f3c6d7b5a5c3b3g5h5g4h4e2g6b6d8c7c8a4a6a7f1a3c2d2b2e1b7g3h3f2d1a1a2b1a8c1g1f7g8e8f8b8g7h8h7h6h2g2h1"
    moves = 'E3F2E2D2E1F6D3D6G4G3G5H3E6H4H6G6F7E7D7F1H5H7G1C8G2F3C6D8E8C5B5F8C7B4B6C3A3B8C2C4B3B2G7A4A1A5B7A2A6D1B1C1H2H8G8H1A8A7'
    moves = 'E6F4C3C4F3D3E3E2E1D2C5G3H3C2D1C1B1B3A3B4G6F2A4B5A5A6A7B6F1G4H4D6C6B2A1A2A8B7B8C8C7D7D8E8E7F8F7G8H8G7H7G5H6H5H2G2G1H1'
    game = othello.OthelloGame(8, 8, 'B', 'W', 'M')

    allmoves = [moves[i:i +2] for i in range(0, len(moves), 2)]
    for move in allmoves:
        # notation is column + row, the game takes them separately
        row = int(move[1])-1
        column = int(cols.index(str.upper(move[0])))
        game.move(row, column)

    print(f'game played, final board: {game.current_board}')


In [ ]:
def run_inference(model_file: str, eval_set: str):

    inference = ClipInference(model_file)

    result = inference.run_eval_file(eval_set)
    print(result)
    for p in result:
        print(result[p][2])

    # test validity of moves
    validity_result = inference.run_validity_check(result)
    print(validity_result)
    for r in validity_result:
        print(f'{validity_result[r][2]},{validity_result[r][3]}')


In [ ]:

class ToTensor:
    # Convert pd series to Tensors
    def __call__(self, sample):

        stringboard = sample['board'].to_list()[0]
        board_array = common.convert_board_array(stringboard)
        board = torch.FloatTensor(board_array)

        move = sample['move'].to_list()[0]
        move = common.convert_move_to_tensor(move)

        return board, move


In [ ]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd

class ClipDataset(Dataset):
    def __init__(self, df: pd.DataFrame):
        self.__df__ = df
        self.length = len(df)
        self.transform = ToTensor()

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        row = self.__df__.iloc[[idx]]

        board, move = self.transform(row)
        item = {}
        item['board'] = board
        item['move'] = move

        return item


In [ ]:
from clip.meter import AvgMeter
from tqdm import tqdm
from data_handling import common

class ClipTrainer:
    '''
    Trains the clip model based on the input variables, parameters and config
    '''

    def __init__(self, source_path: str):
        self.source_path = source_path
        self.source = None

    def __read_source__(self):
        '''
        reads the source data from file into a dataframe
        :return: None
        '''
        self.source = cm.read_dataframe(self.source_path)

    def make_train_validation_sets(self, split=0.2) :
        '''
        creates train and validation dataframes from the source data
        :param split: ration of validation set
        :return: dataframes for train and validation respectively
        '''
        if self.source is None:
            self.__read_source__()

        # there is no 'id' in the dataset, so train / validation split happens on a shuffled dataset
        # for large datasets this can take very long, TODO: optimize if need to rerun often
        df = self.source.sample(frac=1, random_state=42)

        split_index = int((1-split)*len(df))

        train, val = df[:split_index], df[split_index:]
        return train, val

    def getLoader(self, dataframe: pd.DataFrame, mode: str) -> DataLoader:
        '''
        creates a data loader for the frame for training
        :param dataframe:
        :param mode: 'Train' or 'Val'
        :return: torch dataloader
        '''

        ds = ClipDataset(dataframe)
        loader = DataLoader(ds,
                            batch_size=8,
                            num_workers=12,
                            shuffle=True if mode == 'Train' else False,
        )

        return loader


# other static methods for training
def train_epoch(device, model, train_loader, optimizer, lr_scheduler, step):
    loss_meter = AvgMeter()
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:

        batch = {k: v.to(device) for k, v in batch.items()}
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()

        count = batch["board"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=clip.utils.get_lr(optimizer))
    return loss_meter

def valid_epoch(device, model, valid_loader):
    loss_meter = AvgMeter()

    tqdm_object = tqdm(valid_loader, total=len(valid_loader))
    for batch in tqdm_object:
        batch = {k: v.to(device) for k, v in batch.items()}
        loss = model(batch)

        count = batch["board"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(valid_loss=loss_meter.avg)
    return loss_meter

In [ ]:

def train_clip_model(source_file: str):

    trainer = ClipTrainer(source_file)
    df_train, df_val = trainer.make_train_validation_sets()

    # creating data loader
    train_loader = trainer.getLoader(df_train, 'Train')
    val_loader = trainer.getLoader(df_val, 'Val')

    model = CLIPModel().to(utils.get_device())

    print(f'model: {model}')

    optimizer = torch.optim.AdamW(
        #model.parameters(), lr=1e-3, weight_decay=1e-3
        model.parameters(), lr=0.1, weight_decay=1e-3
    )
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", patience=2, factor=0.5
    )
    step = "epoch"

    best_loss = float('inf')

    epochs_count = 6

    for epoch in range(epochs_count):
        print(f"Epoch: {epoch + 1}")
        model.train()
        train_loss = train_epoch(utils.get_device(), model, train_loader, optimizer, lr_scheduler, step)
        model.eval()
        with torch.no_grad():
            valid_loss = valid_epoch(utils.get_device(), model, val_loader)

        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            torch.save(model.state_dict(), os.path.join(data_handling.common.DEFAULT_MODEL_OUTPUT_PATH,"best.pt"))
            print("Saved Best Model!")

In [ ]:
print(utils.get_device())
train_clip_model('/content/clip_training_source')



cuda
model: CLIPModel(
  (image_encoder): ImageEncoder(
    (conv1): Conv2d(2, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (av1): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (fc1): Flatten(start_dim=1, end_dim=-1)
  )
  (text_encoder): TextEncoder()
  (image_projection): ProjectionHead(
    (projection): Linear(in_features=1024, out_features=64, bias=True)
    (gelu): GELU(approximate='none')
    (fc): Linear(in_features=64, out_features=64, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (text_projection): ProjectionHead(
    (projection): Linear(in_featur

100%|██████████| 420238/420238 [29:40<00:00, 236.09it/s, valid_loss=2.08]


Saved Best Model!
Epoch: 2


100%|██████████| 420238/420238 [30:06<00:00, 232.60it/s, valid_loss=2.09]


Epoch: 3


100%|██████████| 420238/420238 [30:14<00:00, 231.55it/s, valid_loss=2.08]


Epoch: 4


 95%|█████████▌| 1603217/1680952 [3:15:38<09:19, 138.94it/s, lr=0.1, train_loss=2.87]